In [1]:
import pandas as pd
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

# Question 1. Select the Tool

**What's the name of the orchestrator you chose?** ```Airflow```

# Question 2. Version

**What's the version of the orchestrator?** 3.0.1

# Question 3. Creating a pipeline

Let's read the March 2023 Yellow taxi trips data.

In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet

--2025-06-09 13:50:03--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.38.147, 18.239.38.83, 18.239.38.181, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.38.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56127762 (54M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2023-03.parquet’

yellow_tripdata_202 100%[===================>]  53.53M   327MB/s    in 0.2s    

2025-06-09 13:50:03 (327 MB/s) - ‘yellow_tripdata_2023-03.parquet’ saved [56127762/56127762]



In [7]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 83.0 MB/s eta 0:00:000m eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
df = pd.read_parquet('yellow_tripdata_2023-03.parquet')

In [4]:
print(df.shape[0])

3403766


How many records did we load?
- 3,003,766
- 3,203,766
- **3,403,766**
- 3,603,766

# Question 4. Data preparation

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df_upd = read_dataframe('yellow_tripdata_2023-03.parquet')

In [7]:
print(df_upd.shape[0])

3316216


**What's the size of the result?**
- 2,903,766
- 3,103,766
- **3,316,216**
- 3,503,766

# Question 5. Train a model

We will now train a linear regression model using the same code as in homework 1.

In [8]:
df_train = df_upd.copy()
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,15.6,3.5,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,7.2,1.0,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333


Fit a dict vectorizer.


In [9]:
categorical = ['PULocationID', 'DOLocationID']
#numerical = ['duration']

df_train[categorical] = df_train[categorical].astype(str)

train_dicts = df_train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

Train a linear regression with default parameters.

In [10]:
target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [15]:
print(lr.intercept_)

24.774419321244913


**What's the intercept of the model?**
- 21.77
- **24.77**
- 27.77
- 31.77

# Question 6. Register the model

The model is trained, so let's save it with MLFlow.

Find the logged model, and find MLModel file. What's the size of the model? (model_size_bytes field):

In [21]:
import mlflow.pyfunc

In [38]:
with mlflow.start_run() as run:
    mlflow.sklearn.log_model(lr, artifact_path="model")

    print("Run ID:", run.info.run_id)
    print("Artifact URI:", mlflow.get_artifact_uri("model"))


2025/06/09 15:07:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Run ID: bc02bea543c74c609dedd5ded020ef11
Artifact URI: file:///workspaces/mlops-zoomcamp/week03/mlruns/0/bc02bea543c74c609dedd5ded020ef11/artifacts/model


In [28]:
!touch MLproject

In [41]:
import yaml
import os

mlmodel_path = "/workspaces/mlops-zoomcamp/week03/mlruns/0/bc02bea543c74c609dedd5ded020ef11/artifacts/model/MLmodel"

with open(mlmodel_path, "r") as f:
    mlmodel = yaml.safe_load(f)

print("✅ Model size in bytes:", mlmodel.get("model_size_bytes"))


✅ Model size in bytes: 4501


**What's the size of the model? (```model_size_bytes``` field):**
- 14,534
- 9,534
- **4,534**
- 1,534